In [8]:
import pandas as pd
import numpy as np

def load_train_test_data(dataset_name, skip_processed = False):
    train_df = pd.read_csv(f"./data/train_data/{dataset_name}.csv")
    y_train = train_df.pop(train_df.columns[-1])
    X_train = train_df

    test_df = pd.read_csv(f"./data/test_data/{dataset_name}.csv")
    y_test = test_df.pop(test_df.columns[-1])
    X_test = test_df

    return X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy()

# def is_already_processed(dataset_path):



# evaluation_table = pd.read_csv('./evaluation_table.csv')['dataset'].unique().tolist()
# evaluation_table
# evaluation_table.columns
# evaluation_table['dataset'].unique().tolist()



['AedesQuinx',
 'AedesSex',
 'ArabicDigit',
 'BNG',
 'HTRU',
 'anuranCalls',
 'click-prediction',
 'cmc',
 'covtype_reduced',
 'default_credit_card_clients',
 'eeg',
 'jm1-processed',
 'letter-recognition',
 'magic',
 'mozilla',
 'namao']